In [2]:
!pip install quandl
import quandl, math
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
token = 'os7VyFRZ9CN2wZ6pZiKB'
dataset = quandl.get("EOD/AAPL", authtoken = token)
dataset.head()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2013-09-03,493.10,500.60,487.35,488.580,11854600.0,0.0,1.0,15.619650,15.857223,15.437510,15.476472,331928800.0
2013-09-04,499.56,502.24,496.28,498.691,12322600.0,0.0,1.0,15.824280,15.909173,15.720381,15.796753,345032800.0
2013-09-05,500.25,500.68,493.64,495.270,8441700.0,0.0,1.0,15.846136,15.859757,15.636755,15.688388,236367600.0
2013-09-06,498.44,499.38,489.95,498.220,12840200.0,0.0,1.0,15.788802,15.818578,15.519869,15.781833,359525600.0
2013-09-09,505.00,507.92,503.48,506.170,12167400.0,0.0,1.0,15.996600,16.089095,15.948451,16.033661,340687200.0


In [7]:
dataset.shape

(1090, 12)

In [8]:
dataset_new = dataset.copy()

dataset_new['HL_PCT'] = (dataset_new['Adj_High'] - dataset_new['Adj_Low'])/dataset_new['Adj_Close'] * 100.0

dataset_new['PCT_Change'] = (dataset_new['Adj_Close'] - dataset_new['Adj_Open'])/dataset_new['Adj_Open'] * 100.0

dataset_new.head()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,HL_PCT,PCT_Change
Date,,,,,,,,,,,,,,
2013-09-03,493.10,500.60,487.35,488.580,11854600.0,0.0,1.0,15.619650,15.857223,15.437510,15.476472,331928800.0,2.711941,-0.916650
2013-09-04,499.56,502.24,496.28,498.691,12322600.0,0.0,1.0,15.824280,15.909173,15.720381,15.796753,345032800.0,1.195129,-0.173953
2013-09-05,500.25,500.68,493.64,495.270,8441700.0,0.0,1.0,15.846136,15.859757,15.636755,15.688388,236367600.0,1.421447,-0.995502
2013-09-06,498.44,499.38,489.95,498.220,12840200.0,0.0,1.0,15.788802,15.818578,15.519869,15.781833,359525600.0,1.892738,-0.044138
2013-09-09,505.00,507.92,503.48,506.170,12167400.0,0.0,1.0,15.996600,16.089095,15.948451,16.033661,340687200.0,0.877176,0.231683


In [10]:
dataset_new_filtered = dataset_new[['Adj_Close','PCT_Change','HL_PCT','Adj_Volume']]
dataset_new_filtered.head()

,Adj_Close,PCT_Change,HL_PCT,Adj_Volume
Date,,,,
2013-09-03,15.476472,-0.916650,2.711941,331928800.0
2013-09-04,15.796753,-0.173953,1.195129,345032800.0
2013-09-05,15.688388,-0.995502,1.421447,236367600.0
2013-09-06,15.781833,-0.044138,1.892738,359525600.0
2013-09-09,16.033661,0.231683,0.877176,340687200.0


In [11]:
dataset_new_filtered.fillna(value = -9999,inplace = True)

forecast_out = int(math.ceil(0.01* len(dataset_new_filtered)))

In [12]:
dataset_new_filtered['label_forecast_out'] = dataset_new_filtered['Adj_Close'].shift(-forecast_out)

In [13]:
dataset_new_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1090 entries, 2013-09-03 to 2017-12-28
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Adj_Close           1090 non-null   float64
 1   PCT_Change          1090 non-null   float64
 2   HL_PCT              1090 non-null   float64
 3   Adj_Volume          1090 non-null   float64
 4   label_forecast_out  1079 non-null   float64
dtypes: float64(5)
memory usage: 51.1 KB


In [14]:
dataset_new_filtered.head(10)

,Adj_Close,PCT_Change,HL_PCT,Adj_Volume,label_forecast_out
Date,,,,,
2013-09-03,15.476472,-0.916650,2.711941,331928800.0,14.719406
2013-09-04,15.796753,-0.173953,1.195129,345032800.0,14.960780
2013-09-05,15.688388,-0.995502,1.421447,236367600.0,14.805882
2013-09-06,15.781833,-0.044138,1.892738,359525600.0,15.541726
2013-09-09,16.033661,0.231683,0.877176,340687200.0,15.492944
2013-09-10,15.668432,-2.283682,3.628902,743195600.0,15.253154
2013-09-11,14.815385,0.149868,1.898612,898696400.0,15.401716
2013-09-12,14.973134,0.894344,1.986503,404051200.0,15.291799
2013-09-13,14.726374,-0.946051,1.533491,298835600.0,15.101740


In [15]:
dataset_new_filtered.dropna(inplace = True)
dataset_new_filtered.isnull().sum

<bound method DataFrame.sum of             Adj_Close  PCT_Change  HL_PCT  Adj_Volume  label_forecast_out
Date                                                                     
2013-09-03      False       False   False       False               False
2013-09-04      False       False   False       False               False
2013-09-05      False       False   False       False               False
2013-09-06      False       False   False       False               False
2013-09-09      False       False   False       False               False
...               ...         ...     ...         ...                 ...
2017-12-06      False       False   False       False               False
2017-12-07      False       False   False       False               False
2017-12-08      False       False   False       False               False
2017-12-11      False       False   False       False               False
2017-12-12      False       False   False       False               False

[1079 

In [18]:
x = np.array(dataset_new_filtered.drop(['label_forecast_out'], axis=1))
y = np.array(dataset_new_filtered['label_forecast_out'])

In [19]:
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
SS.fit(x)
x_scale = SS.transform(x)

In [20]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_scale,y, test_size = 0.2, random_state=2)

In [21]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression(n_jobs = -1)
LR.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [22]:
LR.intercept_

26.486888717018594

In [23]:
print(list(zip(dataset_new_filtered[:-1],LR.coef_)))

[('Adj_Close', 6.359907597505405), ('PCT_Change', -0.04855230707036819), ('HL_PCT', 0.007098791889321637), ('Adj_Volume', -0.10810724422867961)]


In [25]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = LR.predict(x_test)
print("R squared value : ", round(r2_score(y_pred,y_test),4))
print("MSE value ", round(mean_squared_error(y_pred,y_test),4))

R squared value :  0.9637
MSE value  1.3291
